In [29]:
#import libraries
import pandas as pd
import numpy as np

In [30]:
#get the data
data = pd.read_csv("../data/Daily Bike Sharing.csv", 
                   index_col = "dteday", 
                   parse_dates = True)
data.head(1)

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985


In [31]:
#select variables
dataset = data.loc[:, ["cnt", "holiday", "workingday", "weathersit",
                       "temp", "atemp", "hum", "windspeed"]]
dataset.head(1) 

,cnt,holiday,workingday,weathersit,temp,atemp,hum,windspeed
dteday,,,,,,,,
2011-01-01,985,0,0,2,0.344167,0.363625,0.805833,0.160446


In [32]:
#renaming variable
dataset = dataset.rename(columns = {'cnt' : 'y'})
dataset.head(1)

,y,holiday,workingday,weathersit,temp,atemp,hum,windspeed
dteday,,,,,,,,
2011-01-01,985,0,0,2,0.344167,0.363625,0.805833,0.160446


In [33]:
#index
dataset = dataset.asfreq("D")
dataset.index

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10',
               ...
               '2012-12-22', '2012-12-23', '2012-12-24', '2012-12-25',
               '2012-12-26', '2012-12-27', '2012-12-28', '2012-12-29',
               '2012-12-30', '2012-12-31'],
              dtype='datetime64[ns]', name='dteday', length=731, freq='D')

In [34]:
#Stationarity
from statsmodels.tsa.stattools import adfuller
stationarity = adfuller(dataset['y'])
print('Augmented Dickey Fuller p-value: %F' % stationarity[1])

Augmented Dickey Fuller p-value: 0.342743


In [35]:
#Training and test set
test_days = 31
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail(1)

,y,holiday,workingday,weathersit,temp,atemp,hum,windspeed
dteday,,,,,,,,
2012-12-31,2729,0,1,2,0.215833,0.223487,0.5775,0.154846


In [43]:
#exogenous variables
dataset_exog = dataset.iloc[:,1:]
#test_exog = test_set.iloc[:,1:]
#test_exog.head()

In [36]:
#Libraries
from pmdarima import auto_arima

In [41]:
y = dataset['y']
y.shape

(731,)

In [44]:
dataset_exog.shape

(731, 7)

In [45]:
#forecasting model
model = auto_arima(y = dataset['y'],
                   X = dataset_exog,
                   m = 7,
                   seasonal = True,
                   stepwise = False)

In [46]:
#summary
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  731
Model:               SARIMAX(0, 1, 3)   Log Likelihood               -5854.370
Date:                Wed, 26 Jul 2023   AIC                          11732.739
Time:                        13:57:58   BIC                          11787.856
Sample:                    01-01-2011   HQIC                         11754.003
                         - 12-31-2012                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      4.4749      6.726      0.665      0.506      -8.707      17.657
holiday     -201.4847    128.243     -1.571      0.116    -452.837      49.867
workingday    93.9873     59.674      1.575      0.115     -22.971     210.945
weathersit  -483.1694     51.719     -9.342      0.000    -584.536    -381.803
temp        3403.8193   1190.159      2.860      0.004    1071.150    5736.488
atemp       2069.0965   1209.986      1.710      0.087    -302.433    4440.626
hum        -2305.3143    202.271    -11.397      0.000   -2701.758   -1908.871
windspeed  -2317.1408    367.190     -6.310      0.000   -3036.820   -1597.461
ma.L1         -0.5260      0.029    -18.175      0.000      -0.583      -0.469
ma.L2         -0.1361      0.039     -3.476      0.001      -0.213      -0.059
ma.L3         -0.1402      0.041     -3.380      0.001      -0.222      -0.059
sigma2      5.527e+05   2.12e+04     26.024      0.000    5.11e+05    5.94e+05
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):               530.66
Prob(Q):                              0.90   Prob(JB):                         0.00
Heteroskedasticity (H):               2.37   Skew:                            -0.86
Prob(H) (two-sided):                  0.00   Kurtosis:                         6.80
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
#predictions
predictions_sarimax = pd.Series(model.predict(n_periods= test_days,
                              X = test_exog)).rename("SARIMAX")
predictions_sarimax.index = test_set.index                              
predictions_sarimax